In [3]:
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer, HookedTransformerConfig
# from mech_interp_othello_utils import OthelloBoardState
import einops
import torch
from tqdm import tqdm
import numpy as np
from fancy_einsum import einsum
import chess
import numpy as np
import csv

device = "cuda"
# device = "cpu"

n_layers = 16
n_heads = 8
MODEL_DIR = "models/"
DATA_DIR = "data/"
cfg = HookedTransformerConfig(
    n_layers = n_layers,
    d_model = 512,
    d_head = 64,
    n_heads = n_heads,
    d_mlp = 2048,
    d_vocab = 32,
    n_ctx = 1023,
    act_fn="gelu",
    normalization_type="LNPre"
)
model = HookedTransformer(cfg)
model.load_state_dict(torch.load(f'{MODEL_DIR}tf_lens_16.pth'))
model.to(device)

# sd = utils.download_file_from_hf(
#     "NeelNanda/Othello-GPT-Transformer-Lens", "synthetic_model.pth"
# )
# # champion_ship_sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")
# model.load_state_dict(sd)

Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-15): 16 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [4]:
# print(board_seqs_int.shape)
# # print(board_seqs_string.shape)
# dots_indices = torch.tensor(np.load("dots_indices.npy")).long()
# state_stacks = torch.tensor(np.load("state_stacks.npy")).long()

In [5]:
board_seqs_int = torch.tensor(np.load(f"{DATA_DIR}board_seqs_int.npy")).long()
print(board_seqs_int.shape)
dots_indices = torch.tensor(np.load(f"{DATA_DIR}dots_indices.npy")).long()
# state_stack = torch.tensor(np.load("state_stacks_5k.npy")).long()
print(dots_indices.shape)
# print(state_stack.shape)

board_seqs_string = []

with open(f"{DATA_DIR}board_seqs_string.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        board_seqs_string.append(row[0])
print(len(board_seqs_string), len(board_seqs_string[0]))
# print(board_seqs_string[0])


FileNotFoundError: [Errno 2] No such file or directory: 'board_seqs_int.npy'

In [ ]:
def pretty_print_state_stack(state: np.ndarray) -> None:
    """Given a state stack, print each state in a readable format."""
    piece_symbols = {1: 'W', -1: 'B', 0: '.'}

    # Print the rows in reverse order
    for row in reversed(state):
        print(' '.join(piece_symbols[piece] for piece in row))

def board_to_state(board: chess.Board) -> np.ndarray:
    """Given a chess board object, return a 8x8 np.ndarray.
    The 8x8 array should tell if each square is black, white, or blank.
    White is 1, black is -1, and blank is 0.
    In the 8x8 array, row 0 is A1-H1 (White), row 1 is A2-H2, etc."""
    state = np.zeros((8, 8), dtype=int)
    for i in range(64):
        piece = board.piece_at(i)
        if piece:
            # Assign 1 for white pieces and -1 for black pieces
            state[i // 8, i % 8] = 1 if piece.color == chess.WHITE else -1

    return state

def create_state_stack(moves_string: str) -> np.ndarray:
    """Given a string of PGN format moves, create an 8x8 np.ndarray for every character in the string.
    """


    board = chess.Board()
    initial_states = []
    count = 1

    # Scan 1: Creates states, with length = number of moves in the game
    initial_states.append(board_to_state(board))
    # Apply each move to the board
    for move in moves_string.split():
        # because all games are truncated to len 680, often the last move is partial and invalid
        try:
            # Skip move numbers
            if '.' in move:
                board.push_san(move.split(".")[1])
            else:
                board.push_san(move)

            initial_states.append(board_to_state(board))
        except:
            break

        # For debugging and sanity checking
        # count += 1
        # if count % 100 == 0:
        #     pretty_print_state_stack(board_to_state(board))
        #     print("_" * 50)
        #     print(board)

    # return np.array(initial_states)

    # Second Scan: Expand states to match the length of moves_string
    # For ;1.e4 e5 2.Nf3, ";1.e4" = idx 0, " e5" = idx 1, " 2.Nf3" = idx 2
    expanded_states = []
    move_index = 0
    for char in moves_string:
        if char == ' ':
            move_index += 1
        expanded_states.append(initial_states[min(move_index, len(initial_states) - 1)])

    # expanded_states.append(initial_states[-1]) # The last element in expanded_states is the final position of the board.
    # Currently not using this as len(expanded_states) would be 1 greater than len(moves_string) and that would be confusing.
    return np.array(expanded_states)

def create_state_stacks(moves_strings: list[str]) -> torch.Tensor:
    state_stacks = []

    for board in moves_strings:
        state_stack = torch.tensor(create_state_stack(board)).long()
        state_stacks.append(state_stack)

    # Convert the list of tensors to a single tensor
    final_state_stack = torch.stack(state_stacks)
    return final_state_stack

# print(board_seqs_string[0])
print(create_state_stack(board_seqs_string[0]).shape)

state_stack = torch.tensor(create_state_stack(board_seqs_string[0])).long()
print(state_stack.shape)

state_stacks = create_state_stacks(board_seqs_string[:50])
print(state_stacks.shape)

(680, 8, 8)
torch.Size([680, 8, 8])
torch.Size([50, 680, 8, 8])


In [ ]:
layer = 12
batch_size = 1
lr = 2e-4
wd = 0.01
pos_start = 0
# pos_end = model.cfg.n_ctx - 5
input_length = 680
pos_end = input_length - 0
length = pos_end - pos_start
options = 3
rows = 8
cols = 8
num_epochs = 2
num_games = 20000
x = 0
y = 2
probe_name = "main_chess_linear_probe"
# The first mode is blank or not, the second mode is next or prev GIVEN that it is not blank
modes = 1
alternating = torch.tensor([1 if i%2 == 0 else -1 for i in range(length)], device=device)


def state_stack_to_one_hot(state_stack):
    one_hot = torch.zeros(
        modes, # blank vs color (mode)
        state_stack.shape[0], # num games
        state_stack.shape[1], # num moves
        rows, # rows
        cols, # cols
        options, # the two options
        device=state_stack.device,
        dtype=int,
    )
    one_hot[:, ..., 0] = state_stack == 0
    one_hot[:, ..., 1] = state_stack == -1
    one_hot[:, ..., 2] = state_stack == 1
    return one_hot
state_stack_one_hot = state_stack_to_one_hot(state_stacks)
print(state_stack_one_hot.shape)
print((state_stack_one_hot[:, 1, 170, 4:9, 2:5]))
print((state_stacks[1, 170, 4:9, 2:5]))

torch.Size([1, 50, 680, 8, 8, 3])
tensor([[[[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],

         [[0, 1, 0],
          [0, 0, 1],
          [1, 0, 0]],

         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],

         [[0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]]])
tensor([[ 0,  0,  0],
        [-1,  1,  0],
        [ 0,  0,  0],
        [-1,  0,  0]])


In [ ]:
linear_probe = torch.randn(
    modes, model.cfg.d_model, rows, cols, options, requires_grad=False, device=device
)/np.sqrt(model.cfg.d_model)
linear_probe.requires_grad = True
print(linear_probe.shape)
optimiser = torch.optim.AdamW([linear_probe], lr=lr, betas=(0.9, 0.99), weight_decay=wd)

print(dots_indices.shape)
# mask = dots_indices < 245
# dots_indices = dots_indices[mask]

# print(dots_indices.shape)

lr = 3e-4
max_lr = 3e-4
min_lr = lr / 10
max_iters = num_games * num_epochs
decay_lr = True

def get_lr(current_iter: int, max_iters: int, lr: float, min_lr: float) -> float:
    """
    Calculate the learning rate using linear decay.

    Args:
    - current_iter (int): The current iteration.
    - max_iters (int): The total number of iterations for decay.
    - lr (float): The initial learning rate.
    - min_lr (float): The minimum learning rate after decay.

    Returns:
    - float: The calculated learning rate.
    """
    # Ensure current_iter does not exceed max_iters
    current_iter = min(current_iter, max_iters)

    # Calculate the linearly decayed learning rate
    decayed_lr = lr - (lr - min_lr) * (current_iter / max_iters)

    return decayed_lr
current_iter = 0
for epoch in range(num_epochs):
    full_train_indices = torch.randperm(num_games)
    for i in tqdm(range(0, num_games, batch_size)):

        lr = get_lr(current_iter, max_iters, max_lr, min_lr) if decay_lr else lr
        for param_group in optimiser.param_groups:
            param_group['lr'] = lr
        
        indices = full_train_indices[i:i+batch_size]
        list_of_indices = indices.tolist() # For indexing into the board_seqs_string list of strings
        # print(list_of_indices)
        games_int = board_seqs_int[indices]
        games_int = games_int[:, :input_length]
        # print(games_int.shape)
        games_str = [board_seqs_string[idx] for idx in list_of_indices]
        games_str = [s[:input_length] for s in games_str]
        games_dots = dots_indices[indices]
        games_dots = games_dots[:, 5:]
        # print(games_dots.shape)
        state_stack = create_state_stacks(games_str)
        # state_stack = state_stack[:, pos_start:pos_end, :, :]
        # print(state_stack.shape)
        # Initialize a list to hold the indexed state stacks
        indexed_state_stacks = []

        for batch_idx in range(state_stack.size(0)):
            # Get the indices for the current batch
            dots_indices_for_batch = games_dots[batch_idx]

            # Index the state_stack for the current batch
            indexed_state_stack = state_stack[batch_idx, dots_indices_for_batch, :, :]

            # Append the result to the list
            indexed_state_stacks.append(indexed_state_stack)

        # Stack the indexed state stacks along the first dimension
        # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
        state_stack = torch.stack(indexed_state_stacks)
        # print("after indexing state stack shape", state_stack.shape)

        state_stack_one_hot = state_stack_to_one_hot(state_stack).to(device)
        with torch.inference_mode():
            _, cache = model.run_with_cache(games_int.to(device)[:, :-1], return_type=None)
            resid_post = cache["resid_post", layer][:, :]
        # Initialize a list to hold the indexed state stacks
        indexed_resid_posts = []

        for batch_idx in range(games_dots.size(0)):
            # Get the indices for the current batch
            dots_indices_for_batch = games_dots[batch_idx]

            # Index the state_stack for the current batch
            indexed_resid_post = resid_post[batch_idx, dots_indices_for_batch]

            # Append the result to the list
            indexed_resid_posts.append(indexed_resid_post)

        # Stack the indexed state stacks along the first dimension
        # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
        resid_post = torch.stack(indexed_resid_posts)
        # print("Resid post", resid_post.shape)
        probe_out = einsum(
            "batch pos d_model, modes d_model rows cols options -> modes batch pos rows cols options",
            resid_post,
            linear_probe,
        )
        # print(probe_out.shape)

        # acc_blank = (probe_out[0].argmax(-1) == state_stack_one_hot[0].argmax(-1)).float().mean()
        # acc_color = ((probe_out[1].argmax(-1) == state_stack_one_hot[1].argmax(-1)) * state_stack_one_hot[1].sum(-1)).float().sum()/(state_stack_one_hot[1]).float().sum()

        

        probe_log_probs = probe_out.log_softmax(-1)
        probe_correct_log_probs = einops.reduce(
            probe_log_probs * state_stack_one_hot,
            "modes batch pos rows cols options -> modes pos rows cols",
            "mean"
        ) * options # Multiply to correct for the mean over options
        # loss_even = -probe_correct_log_probs[0, 0::2].mean(0).sum() # note that "even" means odd in the game framing, since we offset by 5 moves lol
        # loss_odd = -probe_correct_log_probs[1, 1::2].mean(0).sum()
        loss_all = -probe_correct_log_probs[0, :].mean(0).sum()

        # if i % 1000 == 0:
        #     print(f"epoch {epoch}, batch {i}, acc_blank {acc_blank}, acc_color {acc_color}, loss_even {loss_even}, loss_odd {loss_odd}, loss_all {loss_all}")
        if i % 100 == 0:
            print(f"epoch {epoch}, batch {i}, loss_all {loss_all}, lr {lr}")
        # loss = loss_even + loss_odd + loss_all
        loss = loss_all
        loss.backward() # it's important to do a single backward pass for mysterious PyTorch reasons, so we add up the losses - it's per mode and per square.

        optimiser.step()
        optimiser.zero_grad()
        current_iter += batch_size
torch.save(linear_probe, f"{probe_name}.pth")

torch.Size([1, 512, 8, 8, 3])
torch.Size([41206, 61])


  0%|          | 1/20000 [00:00<53:08,  6.27it/s]

epoch 0, batch 0, loss_all 71.84343719482422, lr 0.0003


  1%|          | 102/20000 [00:11<35:59,  9.21it/s]

epoch 0, batch 100, loss_all 49.125587463378906, lr 0.00029932499999999997


  1%|          | 202/20000 [00:21<36:22,  9.07it/s]

epoch 0, batch 200, loss_all 37.547401428222656, lr 0.00029864999999999997


  2%|▏         | 302/20000 [00:30<33:07,  9.91it/s]

epoch 0, batch 300, loss_all 36.78445053100586, lr 0.00029797499999999997


KeyboardInterrupt: 